In [1]:
# Import necessary modules
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys

from dotenv import load_dotenv
from pathlib import Path
sys.path.insert(0, '..')

from core.genetic_algorithm.ga_bounds import DesignSpaceBounds
from core.genetic_algorithm.ga_params import GeneticAlgorithmParams
from core.genetic_algorithm.ga_run import GeneticAlgorithm

from core.model.model_carrying_capacities import ModelCarryingCapacities
from core.model.model_disturbances import ModelDisturbances
from core.model.model_growth_rates import ModelGrowthRates
from core.model.model_initial_conditions import ModelInitialConditions
from core.model.model_params import ModelParams
from core.model.model_typical_disturbances import ModelTypicalDisturbances

## Environment Variables

In [2]:
load_dotenv(dotenv_path='../core/.env')

True

In [3]:
# General AWS config
ACCOUNT_ID        = os.getenv("ACCOUNT_ID")
ACCESS_KEY_ID     = os.getenv("ACCESS_KEY_ID")
SECRET_ACCESS_KEY = os.getenv("SECRET_ACCESS_KEY")
VPC_ID            = os.getenv("VPC_ID")
SG_ID             = os.getenv("SG_ID")
EC2_PUBLIC_IP     = os.getenv("EC2_PUBLIC_IP")

# Smartfarm-specific AWS config
AWS_REGION    = os.getenv("AWS_REGION")
BUCKET        = os.getenv("BUCKET")
ROOT_PREFIX   = os.getenv("ROOT_PREFIX")
FUNCTION_NAME = os.getenv("FUNCTION_NAME")

## Model Parameters

In [4]:
# Set ModelParams (time stepping)
model_params = ModelParams(
    dt = 0.1, # hours/step
    simulation_hours = 2900 # hours
)

In [5]:
# Set ModelCarryingCapacities
carrying_capacities = ModelCarryingCapacities(
    kh = 3.0,  # m
    kA = 0.65, # m2
    kN = 20,   # number of leaves
    kc = 1000, # number of spikelets
    kP = 0.25  # kg
)

In [6]:
# Set ModelGrowthRates
growth_rates = ModelGrowthRates(
    ah = 0.01,   # 1/hr
    aA = 0.0105, # 1/hr
    aN = 0.011,  # 1/hr
    ac = 0.01,   # 1/hr
    aP = 0.005   # 1/hr
)

In [7]:
# Set ModelInitialConditions
initial_conditions = ModelInitialConditions(
    h0=carrying_capacities.kh/model_params.simulation_hours, # m/hr
    A0=carrying_capacities.kA/model_params.simulation_hours, # m2/hr
    N0=carrying_capacities.kN/model_params.simulation_hours, # number/hr
    c0=carrying_capacities.kc/model_params.simulation_hours, # number/hr
    P0=carrying_capacities.kP/model_params.simulation_hours  # kg/hr
)

## Design Space

In [8]:
lower_bounds_dict = {
    "irrigation_frequency":    1,     # once every X hours
    "irrigation_amount":       0.01,  # inches per irrigation
    "fertilization_frequency": 24,    # once every X hours (once a day)
    "fertilization_amount":    0.001  # pounds per fertilization
}
lower_bounds = np.array([value for key, value in lower_bounds_dict.items()])

In [9]:
upper_bounds_dict = {
    "irrigation_frequency":    24 * 7 * 2,   # once every 2 weeks
    "irrigation_amount":       10,           # inches per irrigation
    "fertilization_frequency": 30 * 24 * 3,  # once every 3 months
    "fertilization_amount":    1             # pounds per fertilization
}
upper_bounds = np.array([value for key, value in upper_bounds_dict.items()])

In [10]:
# Set DesignSpaceBounds
bounds = DesignSpaceBounds(
    lower_bounds=lower_bounds,
    upper_bounds=upper_bounds
)

## Genetic Algorithm Parameters

In [11]:
# Genetic algorithm style inputs
ga_params = GeneticAlgorithmParams(
    num_parents     = 10,  # number of design strings to breed
    num_kids        = 10,  # number of offspring design strings
    num_generations = 200, # number of generations
    num_members     = 100, # number of simulations per generation
    weight_height   = 0.5, # weight for height in cost function
    weight_biomass  = 0.5  # weight for fruit biomass in cost function
)

## Input Disturbances

In [12]:
# Hourly precipitation, radiation, and temperature from CSV
hourly_input_disturbances = pd.read_csv(
    '../io/inputs/hourly_prcp_rad_temp_iowa.csv',
    header=None,
    names=[
        'Hourly Precipitation (mm)',
        'Hourly Radiation (W/m2)',
        'Temperature (C)'
    ]
)

In [13]:
# Set ModelDisturbances
input_disturbances = ModelDisturbances(
    precipitation = hourly_input_disturbances['Hourly Precipitation (mm)'].to_numpy(),
    radiation     = hourly_input_disturbances['Hourly Radiation (W/m2)'].to_numpy(),
    temperature   = hourly_input_disturbances['Temperature (C)'].to_numpy()
)

## Optimal/Typical Values

In [14]:
typical_disturbances = ModelTypicalDisturbances(
        optimal_cumulative_water      = 28,  # inches/acre per season
        optimal_cumulative_fertilizer = 355, # lbs/acre per season
        typical_temperature           = np.mean(input_disturbances.temperature),   # typical temperature over the season
        typical_radiation             = 3/4 * np.max(input_disturbances.radiation) # typical radiation over the season
    )

## Genetic Algorithm

In [15]:
# Create GeneticAlgorithm object from all inputs
GeneticAlgorithm = GeneticAlgorithm(
    bounds               = bounds,
    ga_params            = ga_params,
    model_params         = model_params,
    carrying_capacities  = carrying_capacities,
    growth_rates         = growth_rates,
    initial_conditions   = initial_conditions,
    disturbances         = input_disturbances,
    typical_disturbances = typical_disturbances
)

In [16]:
# Run the Genetic Algorithm and save the result
GeneticAlgorithmResult = GeneticAlgorithm.run()

Generation 0 of 200
Generation 1 of 200
Generation 2 of 200
Generation 3 of 200
RuntimeWarning occurred: overflow encountered in exp


UnboundLocalError: cannot access local variable 'exp_val' where it is not associated with a value